In [ ]:
import threading
import time

def print_numbers():
  for i in range(10):
    print(f"Number {i}")
    time.sleep(1)

def print_letters():
  for letter in 'abcdefghij':
    print(f"Letter {letter}")
    time.sleep(1.5)

# Create threads
t1 = threading.Thread(target=print_numbers)
t2 = threading.Thread(target=print_letters)

# Start threads
t1.start()
t2.start()

# Wait for both threads to finish
t1.join()
t2.join()

print("Both threads are done!")


In [2]:
import tensorflow as tf
print("TensorFlow Version:", tf.__version__)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


TensorFlow Version: 2.10.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15039056368551704947
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 20620212480
locality {
  bus_id: 1
}
incarnation: 4899899610352265151
physical_device_desc: "device: 0, name: DML, pci bus id: <undefined>"
xla_global_id: -1
]


In [4]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

# Regenerate synthetic data
np.random.seed(42)
n_samples = 1000
sqft = np.random.uniform(500, 5000, n_samples)
bedrooms = np.random.randint(1, 6, n_samples)
bathrooms = np.random.randint(1, 5, n_samples)
age = np.random.randint(1, 100, n_samples)
distance_to_city = np.random.uniform(0.1, 20.0, n_samples)
prices = (
    2000 * sqft +
    15000 * bedrooms +
    20000 * bathrooms -
    500 * age -
    1000 * distance_to_city +
    np.random.normal(0, 10000, n_samples)
)
df = pd.DataFrame({
    'sqft': sqft,
    'bedrooms': bedrooms,
    'bathrooms': bathrooms,
    'age': age,
    'distance_to_city': distance_to_city,
    'price': prices
})

# Split the data into training and testing sets
X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Simplified hyperparameter grid
param_dist_simple = {
    'n_estimators': [50, 100],
    'max_depth': [None, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Initialize Randomized Search with 3-fold cross-validation
random_search_simple = RandomizedSearchCV(estimator=RandomForestRegressor(random_state=42), 
                                          param_distributions=param_dist_simple, 
                                          n_iter=16, cv=3, scoring='neg_mean_squared_error', 
                                          verbose=2, random_state=42, n_jobs=-1)

# Perform Randomized Search on the training data
random_search_simple.fit(X_train, y_train)

# Best hyperparameters from simplified Randomized Search
best_params_simple = random_search_simple.best_params_
best_params_simple


Fitting 3 folds for each of 16 candidates, totalling 48 fits


{'n_estimators': 100,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_depth': None}

In [6]:
from sklearn.metrics import mean_squared_error
# Initialize the Random Forest Regressor model with the best hyperparameters
best_rf_model = RandomForestRegressor(
    n_estimators=best_params_simple['n_estimators'],
    min_samples_split=best_params_simple['min_samples_split'],
    min_samples_leaf=best_params_simple['min_samples_leaf'],
    max_depth=best_params_simple['max_depth'],
    random_state=42
)

# Train the model on the training set
best_rf_model.fit(X_train, y_train)

# Predict prices on the testing set
y_test_pred_best_rf = best_rf_model.predict(X_test)

# Calculate the Mean Squared Error on the testing set
mse_test_best_rf = mean_squared_error(y_test, y_test_pred_best_rf)
mse_test_best_rf


1299001043.149296